ДЛЯ КОРРЕКТНОЙ РАБОТЫ ПАРСЕРА НУЖНО ДОБАВИТЬ АКТУАЛЬНЫЕ ССЫЛКИ В ФАЙЛ links.csv В СТОЛБЕЦ "Ссылка"

In [16]:
import requests
import csv
from fake_useragent import UserAgent
from datetime import date
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import warnings
import re 

# from selenium.webdriver.firefox.options import Options 
# import concurrent.futures

warnings.filterwarnings('ignore')

In [17]:
def make_request(link):
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    try:
        url = link
        response = requests.get(url, headers=headers, timeout=30, verify=False)
        parsed = response.json()
        return parsed
    except requests.exceptions.ConnectionError:
        print('Connection error!!!')


def wb_parser(link):
    link = link.replace('https://www.wildberries.ru/catalog/', '')
    if '/' in link:
        index = link.find('/')
        link = link.replace(link[index:], '')
    # try:
    #     parsed = make_request(f"https://card.wb.ru/cards/detail?appType=1&curr=rub&dest=-1257786&regions=80,83,38,4,64,33,68,70,30,40,86,75,69,1,66,110,22,48,31,71,112,114&spp=26&nm={link}")
    # except:
    parsed = make_request(f"https://card.wb.ru/cards/v1/detail?appType=1&curr=rub&dest=-1570684&spp=27&nm={link}")
    try:
        price = parsed['data']['products'][0]['salePriceU'] / 100
    except:
        try:
            sizes = ['17.5', '17', '18', '19', '45', '50']
            for i in range(len(parsed['data']['products'][0]['sizes'])):
                size = parsed['data']['products'][0]['sizes'][i]['origName']
                if size in sizes:
                    price = parsed['data']['products'][0]['sizes'][i]['salePriceU'] / 100
                    break

        except:
            try:
                for i in range(len(parsed['data']['products'])):
                    if parsed['data']['products'][i]['id'] == int(link):
                        price = parsed['data']['products'][i]['salePriceU'] / 100
                        break
            except:
                price = np.nan     
    price = str(price)
    price = price.replace('.', ',')      
    discount = 1
    product = [price, discount]
    return product       
    

In [18]:
def sunlight_parser(link):
    url = link
    for i in link:
        try:
            int(i)
            id = link.index(i)
            link = link[id:]
            id_ = link.index('.')
            link = link[:id_]
            break
        except:
            continue

    cookies = {
        'city_id': '118',
        'city_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
        'city_full_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
        'region_id': '090a0c31-3b91-49a3-aa35-0c784898170b',
        'region_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
        'region_subdomain': 'spb',
        'tmr_lvid': '5f6ab165f41caef626387d8e9d8799df',
        'tmr_lvidTS': '1697011590764',
        '_ym_uid': '1697011591885937639',
        '_ym_d': '1697011591',
        'qr-go-to-the-app-banner': '1',
        '_tt_enable_cookie': '1',
        '_ttp': 'fKiEqa8XyglrSfN_aUZvbCNG6ew',
        'ccart': 'off',
        'uxs_uid': '1766f130-680d-11ee-8541-1b4e2f56831c',
        '_gid': 'GA1.2.1640259099.1697631825',
        'city_auto_popup_shown': '1',
        '_ym_isad': '2',
        'session_id': 'e85480a6-9ba8-4772-8afa-57a70a2eafff',
        '_gat_UA-11277336-11': '1',
        '_gat_UA-11277336-12': '1',
        '_ga_HJNSJ6NG5J': 'GS1.1.1697719835.7.0.1697719835.60.0.0',
        'digi_uc': 'W1sidiIsIjg4NDExIiwxNjk3NzE5ODM1OTE5XSxbInYiLCI3MTM4OSIsMTY5NzcwNjEzNzY3Ml0sWyJ2IiwiMjc1NjE1IiwxNjk3NzA2MTEyODk0XSxbInYiLCIzNDI5MDIiLDE2OTc3MDI0ODkyMDVdLFsidiIsIjI3MzkxOSIsMTY5NzcwMjQ3NTU1MV0sWyJ2IiwiMTUyNjA5IiwxNjk3NzAyNDAwMTcwXSxbInYiLCI1MzUzNiIsMTY5NzcwMjM1ODE2NF0sWyJ2IiwiMTUyNjEyIiwxNjk3NzAyMzEzNzI5XSxbInYiLCIyNjI1MzkiLDE2OTc3MDIyOTc4NTRdLFsidiIsIjE1MTk5NiIsMTY5NzcwMjI3ODU1NF1d',
        '_ga': 'GA1.2.1274284271.1697011591',
        '_ga_GRSMPDQ6SE': 'GS1.2.1697719836.7.0.1697719836.60.0.0',
        '_ga_M9ZHPEJXBM': 'GS1.2.1697719836.7.0.1697719836.60.0.0',
        '_ym_visorc': 'b',
        'mindboxDeviceUUID': '6e9424e5-a80e-43b6-b480-b1d86e4084fd',
        'directCrm-session': '%7B%22deviceGuid%22%3A%226e9424e5-a80e-43b6-b480-b1d86e4084fd%22%7D',
    }
    headers = {
        'AB-TESTS': '{"personal_feed":"cumulative"}',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/json',
        # 'Cookie': 'city_id=118; city_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; city_full_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_id=090a0c31-3b91-49a3-aa35-0c784898170b; region_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_subdomain=spb; tmr_lvid=5f6ab165f41caef626387d8e9d8799df; tmr_lvidTS=1697011590764; _ym_uid=1697011591885937639; _ym_d=1697011591; qr-go-to-the-app-banner=1; _tt_enable_cookie=1; _ttp=fKiEqa8XyglrSfN_aUZvbCNG6ew; ccart=off; uxs_uid=1766f130-680d-11ee-8541-1b4e2f56831c; _gid=GA1.2.1640259099.1697631825; city_auto_popup_shown=1; _ym_isad=2; session_id=e85480a6-9ba8-4772-8afa-57a70a2eafff; _gat_UA-11277336-11=1; _gat_UA-11277336-12=1; _ga_HJNSJ6NG5J=GS1.1.1697719835.7.0.1697719835.60.0.0; digi_uc=W1sidiIsIjg4NDExIiwxNjk3NzE5ODM1OTE5XSxbInYiLCI3MTM4OSIsMTY5NzcwNjEzNzY3Ml0sWyJ2IiwiMjc1NjE1IiwxNjk3NzA2MTEyODk0XSxbInYiLCIzNDI5MDIiLDE2OTc3MDI0ODkyMDVdLFsidiIsIjI3MzkxOSIsMTY5NzcwMjQ3NTU1MV0sWyJ2IiwiMTUyNjA5IiwxNjk3NzAyNDAwMTcwXSxbInYiLCI1MzUzNiIsMTY5NzcwMjM1ODE2NF0sWyJ2IiwiMTUyNjEyIiwxNjk3NzAyMzEzNzI5XSxbInYiLCIyNjI1MzkiLDE2OTc3MDIyOTc4NTRdLFsidiIsIjE1MTk5NiIsMTY5NzcwMjI3ODU1NF1d; _ga=GA1.2.1274284271.1697011591; _ga_GRSMPDQ6SE=GS1.2.1697719836.7.0.1697719836.60.0.0; _ga_M9ZHPEJXBM=GS1.2.1697719836.7.0.1697719836.60.0.0; _ym_visorc=b; mindboxDeviceUUID=6e9424e5-a80e-43b6-b480-b1d86e4084fd; directCrm-session=%7B%22deviceGuid%22%3A%226e9424e5-a80e-43b6-b480-b1d86e4084fd%22%7D',
        'Origin': 'https://spb.sunlight.net',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
        'X-Requested-With': 'SunlightFrontendApp',
        'sec-ch-ua': '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }
    params = {
        'article': f'{link}',
    }
    
    params1 = {
    'article': f'{link}',
    'city_id': '118',
    }   

    response1 = requests.get(
        'https://api.sunlight.net/v6/catalog/products/card/purchase/',
        params=params1,
        cookies=cookies,
        headers=headers,
    )

    response = requests.get('https://api.sunlight.net/v6/catalog/products/card/prices/',
        params=params,
        cookies=cookies,
        headers=headers,
    )
    discounts = response1.json()
    data = response.json()

    try:
        price = data['content']['sizes']['17.50']['result_sum']
        discount = discounts['content']['sizes']['17.50']['nameplates'][0]['text']['label']
    except:
        try:
            price = data['content']['sizes']['17.00']['result_sum']
            discount = discounts['content']['sizes']['17.00']['nameplates'][0]['text']['label']
        except:
            try:
                price = data['content']['sizes']['19.00']['result_sum']
                discount = discounts['content']['sizes']['19.00']['nameplates'][0]['text']['label']
            except:
                try:
                    price = data['content']['sizes']['45.00']['result_sum']
                    discount = discounts['content']['sizes']['45.00']['nameplates'][0]['text']['label']
                except:
                    try:
                        price = data['content']['sizes']['50.00']['result_sum']
                        discount = discounts['content']['sizes']['50.00']['nameplates'][0]['text']['label']
                    except:
                        try:
                            price = data['content']['sizes']['18.00']['result_sum']
                            discount = discounts['content']['sizes']['18.00']['nameplates'][0]['text']['label']
                        except:
                            try:
                                price = data['content']['sizes']['0.00']['result_sum']
                                discount = discounts['content']['sizes']['0.00']['nameplates'][0]['text']['label']
                            except:
                                try:
                                    browser = webdriver.Firefox()
                                    browser.get(url)
                                    time.sleep(2)
                                    article = browser.find_element(By.CSS_SELECTOR, 'div[class="supreme-product-card__product-article-content"]').text.strip()
                                    discount = browser.find_element(By.CSS_SELECTOR, 'span[class="supreme-product-card-nameplates__item"]').text.strip()
                                    nums = re.findall(r'\d+', article)
                                    nums = [int(i) for i in nums]
                                    article = nums[0]
                                    params = {'article': f'{article}'}
                                    response = requests.get('https://api.sunlight.net/v6/catalog/products/card/prices/', params=params, cookies=cookies, headers=headers, timeout=30)
                                    data = response.json()
                                    try:
                                        price = data['content']['sizes']['17.00']['result_sum']
                                    except:
                                        price = data['content']['sizes']['0.00']['result_sum']
                                    browser.close()
                                except:
                                    price = np.nan
                                    discount = np.nan
    price = price.replace(u'\u202f', '').replace('₽', '').replace(' ', '').replace(u'\xa0', '')
    try:
        nums = re.findall(r'\d+', discount)
        nums = [int(i) for i in nums]
        discount = nums[0]
        discount =  str(1 - discount / 100)
        discount = discount.replace('.', ',')
    except:
        discount = 1 
    product = [price, discount]
    return product


In [19]:
def sokolov_parser(link):
    cookies = {
    '_userGUID': '0:lnvpvltk:3jPTv4pojH2Pph6FbhhwQgSmBuQf0U3y',
    'guid_city': '0c5b2444-70a0-4932-980c-b4dc0d3f02b5',
    'name_city': '%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0',
    'guid_region': '0c5b2444-70a0-4932-980c-b4dc0d3f02b5',
    'guid_country': '8aa15da9-92a4-4530-ab74-1992c973c539',
    'region_timezone': 'UTC%2B3%3A00',
    'ab-test-user-id': '63cd24c2a743bb2433f8110f500a594d',
    'fuser_id': 'fa4970464c86b53f66816e5840afc71b5c98b495c898a2314d76b2aaf748bba8a%3A2%3A%7Bi%3A0%3Bs%3A8%3A%22fuser_id%22%3Bi%3A1%3Bs%3A32%3A%22fcb2d248286d2f75ced78b87a0bbd8f2%22%3B%7D',
    '_gcl_au': '1.1.1753721034.1697702901',
    '_ym_uid': '1697702901888153736',
    '_ym_d': '1697702901',
    'flocktory-uuid': '88979721-0c4b-427b-87fb-a99179dc480c-4',
    'tmr_lvid': 'ce6c20b9a094b1708f0b7d19879d76d2',
    'tmr_lvidTS': '1697702901032',
    'mindboxDeviceUUID': '6e9424e5-a80e-43b6-b480-b1d86e4084fd',
    'directCrm-session': '%7B%22deviceGuid%22%3A%226e9424e5-a80e-43b6-b480-b1d86e4084fd%22%7D',
    'syte_uuid': 'ab6463f0-6e56-11ee-b1f4-bd5f3a332347',
    'inova_p_sid': 'uek8p4e_231019010820',
    'adrcid': 'ANIHYs14bKTowSR_pRsEFrg',
    'popmechanic_sbjs_migrations': 'popmechanic_1418474375998%3D1%7C%7C%7C1471519752600%3D1%7C%7C%7C1471519752605%3D1',
    'uxs_uid': 'ad56e520-6e56-11ee-a612-3bba5efe5043',
    'syte_ab_tests': '{}',
    'theme-mode': 'light',
    'dSesn': '7751f85a-b31b-a629-9e48-85f7f8907993',
    '_dvs': '0:lpckbaad:bmuIGxk6bJKfbJGyipoDzvEpK8r1uzm7',
    'iso_cookie': 'undefined',
    'UX_utm_source': 'undefined',
    'UX_utm_medium': 'undefined',
    '_gid': 'GA1.2.692823511.1700826728',
    '_ym_isad': '1',
    'stimgs': '{%22sessionId%22:80884514%2C%22didReportCameraImpression%22:false%2C%22newUser%22:false}',
    '_ym_visorc': 'b',
    'sklvt': '$2y$10$DvaTYSqhGwIZ/U.uRxe3AOhyH/Hb2gB3YUA7rIAH/poMLjXlfBspO',
    'PHPSESSID': 'k7g7cji0t6vns3ufp0b2ch8o1n',
    '_gat_UA-50519746-1': '1',
    'hits_count': '5',
    'tmr_detect': '1%7C1700827425715',
    '_ga': 'GA1.1.230823214.1697702901',
    '_ga_WPBGN0ND17': 'GS1.1.1700826727.13.1.1700827442.0.0.0',
    '_ga_MZNG6JMV9K': 'GS1.1.1700826727.13.1.1700827442.36.0.0',
    '_ga_VRRLSXGS0X': 'GS1.1.1700827422.6.0.1700827442.0.0.0',
    '_csrf': 'b3b85ab2972487aa943f078802eada3e00c1ff13f7ce07f56f1db15fa6404935a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22NitUVqyphirYsvGBrENDueRb6tfl38sA%22%3B%7D',
}

    headers = {
    'Accept': 'application/json',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    # 'Cookie': '_userGUID=0:lnvpvltk:3jPTv4pojH2Pph6FbhhwQgSmBuQf0U3y; guid_city=0c5b2444-70a0-4932-980c-b4dc0d3f02b5; name_city=%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0; guid_region=0c5b2444-70a0-4932-980c-b4dc0d3f02b5; guid_country=8aa15da9-92a4-4530-ab74-1992c973c539; region_timezone=UTC%2B3%3A00; ab-test-user-id=63cd24c2a743bb2433f8110f500a594d; fuser_id=fa4970464c86b53f66816e5840afc71b5c98b495c898a2314d76b2aaf748bba8a%3A2%3A%7Bi%3A0%3Bs%3A8%3A%22fuser_id%22%3Bi%3A1%3Bs%3A32%3A%22fcb2d248286d2f75ced78b87a0bbd8f2%22%3B%7D; _gcl_au=1.1.1753721034.1697702901; _ym_uid=1697702901888153736; _ym_d=1697702901; flocktory-uuid=88979721-0c4b-427b-87fb-a99179dc480c-4; tmr_lvid=ce6c20b9a094b1708f0b7d19879d76d2; tmr_lvidTS=1697702901032; mindboxDeviceUUID=6e9424e5-a80e-43b6-b480-b1d86e4084fd; directCrm-session=%7B%22deviceGuid%22%3A%226e9424e5-a80e-43b6-b480-b1d86e4084fd%22%7D; syte_uuid=ab6463f0-6e56-11ee-b1f4-bd5f3a332347; inova_p_sid=uek8p4e_231019010820; adrcid=ANIHYs14bKTowSR_pRsEFrg; popmechanic_sbjs_migrations=popmechanic_1418474375998%3D1%7C%7C%7C1471519752600%3D1%7C%7C%7C1471519752605%3D1; uxs_uid=ad56e520-6e56-11ee-a612-3bba5efe5043; syte_ab_tests={}; theme-mode=light; dSesn=7751f85a-b31b-a629-9e48-85f7f8907993; _dvs=0:lpckbaad:bmuIGxk6bJKfbJGyipoDzvEpK8r1uzm7; iso_cookie=undefined; UX_utm_source=undefined; UX_utm_medium=undefined; _gid=GA1.2.692823511.1700826728; _ym_isad=1; stimgs={%22sessionId%22:80884514%2C%22didReportCameraImpression%22:false%2C%22newUser%22:false}; _ym_visorc=b; sklvt=$2y$10$DvaTYSqhGwIZ/U.uRxe3AOhyH/Hb2gB3YUA7rIAH/poMLjXlfBspO; PHPSESSID=k7g7cji0t6vns3ufp0b2ch8o1n; _gat_UA-50519746-1=1; hits_count=5; tmr_detect=1%7C1700827425715; _ga=GA1.1.230823214.1697702901; _ga_WPBGN0ND17=GS1.1.1700826727.13.1.1700827442.0.0.0; _ga_MZNG6JMV9K=GS1.1.1700826727.13.1.1700827442.36.0.0; _ga_VRRLSXGS0X=GS1.1.1700827422.6.0.1700827442.0.0.0; _csrf=b3b85ab2972487aa943f078802eada3e00c1ff13f7ce07f56f1db15fa6404935a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22NitUVqyphirYsvGBrENDueRb6tfl38sA%22%3B%7D',
    'Referer': 'https://sokolov.ru/jewelry-catalog/product/140006/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'X-City-Uuid': '0c5b2444-70a0-4932-980c-b4dc0d3f02b5',
    'X-Country-Uuid': '8aa15da9-92a4-4530-ab74-1992c973c539',
    'X-Region-Uuid': '0c5b2444-70a0-4932-980c-b4dc0d3f02b5',
    'X-Source': 'site',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}
    link = link.replace('https://sokolov.ru/jewelry-catalog/product/', '')
    link = link.replace('/', '')
    response = requests.get(f'https://catalog.sokolov.ru/api/v2/products/{link}/', headers=headers, cookies=cookies, timeout=30, verify=False)
    product = response.json()
    url = product['url_code']
    # time.sleep(0.5)
    item = requests.get(f'https://sokolov.ru/api/v4/catalog/product-info/{url}/', headers=headers, timeout=30, verify=False, cookies=cookies)
    item = item.json()
    try:
        price = item['price'][1]['value']
    except:
        price = item['price'][0]['value']
    # time.sleep(0.5)
    item = requests.get(f'https://sokolov.ru/api/v5/products/coupon/{url}/', headers=headers, cookies=cookies, timeout=30, verify=False)
    item = item.json()
    try:
        discount = str((100 - item['sale_value']) / 100)
        discount = discount.replace('.', ',')
        #discount = item['detail_description']
    except:
        discount = 1
    product = [price, discount]
    return product


ПЕРЕД ПАРСИНГОМ ОЗОНА ВСЕГДА ПРОВЕРЯТЬ НАЗВАНИЕ КЛАССА В КАРТОЧКЕ ТОВАРА "ЦЕНА С ОЗОН КАРТОЙ"

In [20]:
def ozon_parser(link):
    browser = webdriver.Firefox()
    browser.get(link)
    time.sleep(1)
    try:
        try:
            price = browser.find_element(By.CSS_SELECTOR, 'span[class="ol9 o7l"]').text.strip() 
        except:
            price = browser.find_element(By.CSS_SELECTOR, 'span[class="p3l lp4 l8p"]').text.strip() 
        price = price.replace('₽', '')
    except:
        price = np.nan    
    price = price.replace(u'\xa0',' ').replace(u'\u2009', '')
    discount = 1
    product = [price, discount]
    browser.close()
    return product    

ДАННЫЕ ПО НАШЕМУ МАГАЗИНУ СОБИРАЮТСЯ С АКТУАЛЬНОЙ ВЫГРУЗКИ ЕКОМА, ПЕРЕД АКТУАЛЬНЫМ ПАРСИНГОМ УКАЗАТЬ АКТУАЛЬНЫЙ ПУТЬ ДО АКТУАЛЬНОГО ФАЙЛА

In [21]:
# читаю файл для получения списка ссылок
urls = pd.read_csv(r'C:\Users\Trenkin.Sergey\Desktop\trenkinsv\проекты\мониторинг индикаторов\universal_parser\parser\links.csv')
urls = urls['Ссылка'].tolist()

# из ecom-файла достаю все данные и сохраняю их в DataFrame
df1 = pd.read_csv(r'\\gold585.int\uk\Общее хранилище файлов\Дирекция Маркетинга и организации розничных продаж\Отдел аналитики\Выгрузки из БД\2023-11-27_остатки сайта.csv') # сюда
df1 = df1.drop_duplicates()

# вытаскиваю только наши ссылки для дальнейшего мониторинга
links = []
for url in urls:
    try:
        if '585zolotoy' in url:
            links.append(url)
    except:
        continue

# извлекаю из ссылок артикулы
urls = []
for link in links:
    link = link.replace('https://www.585zolotoy.ru/catalog/products/', '')
    if '/' in link:
        link = link.replace('/', '')
    urls.append(int(link))

# немного фильтрации по отслеживаемым позициям
b = df1[(df1['article'] == 1001247)]
df = df1.query("article.isin(@urls)")
rings = df[((df['type1'] == 'КОЛЬЦО ОБРУЧАЛЬНОЕ') & (df['size'] == 17.5)) | ((df['type1'] == 'КОЛЬЦО') & (df['size'] == 17))].drop_duplicates(['article'])
chains = df[(df['type1'] == 'ЦЕПЬ') & (df['size'] == 45)].drop_duplicates(['article'])
bracelets = df[(df['type1'] == 'БРАСЛЕТ') & (df['size'] == 18)].drop_duplicates(['article'])
earrings = df[(df['type1'] == 'СЕРЬГИ')].drop_duplicates(['article'])
earrings_kongo = df[((df['type1'] == 'СЕРЬГИ-КОНГО') & (df['size'] == 1.5))]
other = df['type1'][(df['type1'] != 'КОЛЬЦО ОБРУЧАЛЬНОЕ') &
                    (df['type1'] != 'КОЛЬЦО') &
                    (df['type1'] != 'ЦЕПЬ') &
                    (df['type1'] != 'БРАСЛЕТ') &
                    (df['type1'] != 'СЕРЬГИ') &
                    (df['type1'] != 'СЕРЬГИ-КОНГО')
                    ].unique().tolist()
df = df.query("type1.isin(@other)")
df = df.drop_duplicates(['article'])

# формирую итоговый DataFrame с товарами-индикаторами 
res = pd.concat([df, rings, chains, bracelets, earrings, earrings_kongo, b], axis=0)
res = res[['article', 'size', 'discount_price', 'label_template']]

In [22]:
# преобразую столбец со скидками в числовой формат
def discount(price):
    try:
        if '-30%' in price:
            return 0.70
        elif '-10%' in price:
            return 0.90
        elif '-15%' in price:
            return 0.85
        return 0
    except:
        return 1
    
res['label_template'] = res['label_template'].apply(discount)

In [23]:
links = pd.read_csv(r'C:\Users\Trenkin.Sergey\Desktop\trenkinsv\проекты\мониторинг индикаторов\universal_parser\parser\links.csv')
links = links['Ссылка'].tolist()
products = []

In [24]:
for link in tqdm(links):
    try:
        if 'ozon.ru' in link:
            products.append(ozon_parser(link))
        elif 'wildberries.ru' in link:
            products.append(wb_parser(link))
        elif '585zolotoy.ru' in link:
            link = link.replace('https://www.585zolotoy.ru/catalog/products/', '')
            if '/' in link:
                link = link.replace('/', '')
            price = res[res['article'] == int(link)]['discount_price'].mean()
            price *= 0.95
            price = str(price)
            price = price.replace('.', ',')
            discount = res[res['article'] == int(link)]['label_template'].mean()
            discount = str(discount)
            discount = discount.replace('.', ',')
            products.append([price, discount])
        elif 'sokolov.ru' in link:
            products.append(sokolov_parser(link))
        elif 'sunlight' in link:
            products.append(sunlight_parser(link))
    except:
        products.append([np.nan, np.nan])

  0%|          | 0/239 [00:00<?, ?it/s]

3800,0 0,7


  2%|▏         | 5/239 [00:10<06:24,  1.64s/it]

12464,0 1,0


In [93]:
date = date.today()
with open(f"actual_{date}.csv", mode="w", encoding='utf-8') as file:
    file_writer = csv.writer(file, delimiter=",", lineterminator="\n")
    file_writer.writerow(['price', 'discount'])
    for i in range(0, len(products)):
        try:
            rows = products[i]
            file_writer.writerow(rows)
        except:
            pass

In [94]:
# запускать, если нужны данные в Excel формате
parser = pd.read_csv(f'actual_{date}.csv')
parser.to_excel(f'actual_{date}.xlsx')

In [57]:
# def main(url):
#     try:
#         if 'ozon.ru' in url:
#             return ozon_parser(url)
#         elif 'wildberries.ru' in url:
#             return wb_parser(url)
#         elif '585zolotoy.ru' in url:
#             return ecom_parser(url)
#         elif 'sokolov.ru' in url:
#             return sokolov_parser(url)
#         elif 'sunlight' in url:
#             return sunlight_parser(url)
#     except:
#         return [np.nan, np.nan]

In [ ]:
# CONNECTIONS = 6

# with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
#     future_to_url = (executor.submit(main, url) for url in links)
#     for future in tqdm(concurrent.futures.as_completed(future_to_url), total=len(links)):
#         try:
#             data = future.result() 
#         except Exception as exc:      
#             data = str(type(exc))
#         finally:
#             products.append(data)